<a href="https://colab.research.google.com/github/Yosingh1/Machine_Learning/blob/main/Summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Sat Feb 24 07:20:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import transformers

In [3]:
!pip install transformers[sentencepiece]

In [4]:
!pip install sacrebleu

In [5]:
!pip install rouge_score

In [6]:
!pip install py7zr #unzipping the file

In [7]:
!pip install accelerate

In [8]:
import torch

In [57]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [58]:
device

'cuda'

In [11]:
import pandas as pd
import nltk
from tqdm import tqdm
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq

In [13]:
!pip install datasets

In [14]:
from datasets import load_dataset, load_from_disk, load_metric

In [15]:
model= "google/pegasus-cnn_dailymail"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
pegasus_model= AutoModelForSeq2SeqLM.from_pretrained(model).to("cuda")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#traning of model (finetunig)
#prediction

In [19]:
data= load_dataset("samsum")

In [20]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [21]:
print(data["train"]["dialogue"][0])

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)


In [22]:
print(data["train"]["summary"][0])

Amanda baked cookies and will bring Jerry some tomorrow.


In [23]:
def convert_example_to_features(data_in_batch):
  input_encoding= tokenizer(data_in_batch["dialogue"], max_length=1024,truncation=True)


  target_encoding=tokenizer(data_in_batch["summary"], max_length=128,truncation=True)

  return {"input_ids":input_encoding["input_ids"],
          "attention_mask":input_encoding["attention_mask"],
          "labels":target_encoding["input_ids"]}

In [24]:
data=data.map(convert_example_to_features, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [25]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [26]:
data["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [27]:
data["train"]["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [28]:
#finetuning

In [29]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [30]:
from transformers import TrainingArguments, Trainer

In [31]:
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [32]:
training_args=TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [33]:
trainer=Trainer(model=pegasus_model,args=training_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                train_dataset=data["test"], #pass test data becuase training data was hug and it could have taken more time
                eval_dataset=data["validation"])

In [34]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 179.0155, 'train_samples_per_second': 4.575, 'train_steps_per_second': 0.285, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 1.0})

In [59]:
# evalation of model
# Evaluation

In [60]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [61]:
rouge_names=["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-61-582344e78217>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [63]:
score = calculate_metric_on_test_ds(
    data['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:18<00:00,  3.71s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022145,0.0,0.02181,0.022025


In [40]:
pegasus_model.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [35]:
tokenizer.save_pretrained("samsum-tokenizer")

('samsum-tokenizer/tokenizer_config.json',
 'samsum-tokenizer/special_tokens_map.json',
 'samsum-tokenizer/spiece.model',
 'samsum-tokenizer/added_tokens.json',
 'samsum-tokenizer/tokenizer.json')

In [36]:
tokenizer=AutoTokenizer.from_pretrained("/content/samsum-tokenizer")

In [41]:
model=AutoModelForSeq2SeqLM.from_pretrained("/content/pegasus-samsum-model").to('cuda')

In [44]:
sample_text_for_text=data["train"]["dialogue"][2]

In [56]:
print(sample_text_for_text)

Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban style


In [45]:
summary=data["train"]["summary"][2]

In [50]:
summary

'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'

In [51]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

In [46]:
pipeline_object=pipeline("summarization",model=model,tokenizer=tokenizer)

In [52]:
pipeline_object(sample_text_for_text,**gen_kwargs)

[{'summary_text': "Kim was going to do lots of stuff but ended up procrastinating .<n>We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies .<n>For doing stuff I recommend Pomodoro technique where u use breaks for doing chores ."}]

In [53]:
prediction=pipeline_object(sample_text_for_text)[0]["summary_text"]

In [54]:
print(prediction)

Kim was going to do lots of stuff but ended up procrastinating .<n>We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies .<n>For doing stuff I recommend Pomodoro technique where u use breaks for doing chores .
